In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


# Dynamic Graphs

In [3]:
node_list = np.array(list(g.nodes.keys()))
# the i-th node in node_list corresponds to the i-th row of Adjacency matrix

# from node to index in node_list
node_list_index_dict = {}
for node in g.nodes:
    node_list_index_dict[node] = np.where(node_list == node)[0][0]

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]

In [5]:
top_prop = 0.5

adjacency_list = []

A = nx.adjacency_matrix(g).todense()
A_init = A.copy()

for rep in tqdm(range(len(ramps))):
    np.random.seed(0)
    
    degrees = A.sum(axis=1)
    deg_bar = np.quantile(degrees, 1-top_prop)
    evolve_nodes_index = np.where(degrees >= deg_bar)[0]
    deg_prob = degrees / degrees.sum()
    
    for i in evolve_nodes_index:    
        connect_node_indices = np.random.choice(range(num_nodes), size = 5, p=deg_prob)
        
        for connect_node_idx in connect_node_indices:
            # i_node_key = node_list[i]
            # conn_node_key = node_list[connect_node_idx]
            i_node_key = i 
            conn_node_key = connect_node_idx
            
            if i_node_key == conn_node_key:
                continue
            if A[i_node_key][conn_node_key] == 0:
                A[i_node_key][conn_node_key] = 1
                A[conn_node_key][i_node_key] = 1
            
    adjacency_list.append(A.copy())

adjacency_list.append(A_init)

100%|██████████| 5/5 [00:22<00:00,  4.42s/it]


In [6]:
g_list = [nx.from_numpy_array(adj_mat) for adj_mat in adjacency_list]

In [7]:
for adj_mat in adjacency_list:
    print(adj_mat.sum()//2)

596414
621807
648583
671663
698233
568309


# Model Training

In [8]:
dgl_G_list = [dgl.from_networkx(g_) for g_ in g_list]

In [13]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
ramps_add = ramps + [0]
repeat_num = 1000

In [14]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):
    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")        
        torch.manual_seed(1)            
        g_feat_list = []
        y_list = []        


        for step_idx in [len(ramps), num_step]: # global control, step t
            g = g_list[step_idx]
            dgl_G = dgl_G_list[step_idx]
            p = ramps_add[step_idx]
            
            z = (rollout_index < np.quantile(rollout_index, p))
            
            nx.set_node_attributes(g, 0, "z")
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")
            
            po_linear_model(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.degree[n]] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(2):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {ramps[num_step]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/Dynamic_incre_CR_unit.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.02, Loss: 0.3663
Epoch 0, Treat Prop: 0.02, Loss: 0.2253
Epoch 50, Treat Prop: 0.02, Loss: 0.0150
Epoch 50, Treat Prop: 0.02, Loss: 0.0303
Epoch 100, Treat Prop: 0.02, Loss: 0.0144
Epoch 100, Treat Prop: 0.02, Loss: 0.0213
Epoch 150, Treat Prop: 0.02, Loss: 0.0152
Epoch 150, Treat Prop: 0.02, Loss: 0.0147
Epoch 200, Treat Prop: 0.02, Loss: 0.0142
Epoch 200, Treat Prop: 0.02, Loss: 0.0134
Epoch 250, Treat Prop: 0.02, Loss: 0.0104
Epoch 250, Treat Prop: 0.02, Loss: 0.0121
Epoch 300, Treat Prop: 0.02, Loss: 0.0102
Epoch 300, Treat Prop: 0.02, Loss: 0.0120
Epoch 350, Treat Prop: 0.02, Loss: 0.0101
Epoch 350, Treat Prop: 0.02, Loss: 0.0132
0.012553862296044827
Seed: 0
Epoch 0, Treat Prop: 0.05, Loss: 0.3644
Epoch 0, Treat Prop: 0.05, Loss: 0.2698
Epoch 50, Treat Prop: 0.05, Loss: 0.0181
Epoch 50, Treat Prop: 0.05, Loss: 0.0550
Epoch 100, Treat Prop: 0.05, Loss: 0.0154
Epoch 100, Treat Prop: 0.05, Loss: 0.0280
Epoch 150, Treat Prop: 0.05, Loss: 0.0130
Epoch 150, Treat 

  0%|          | 1/1000 [00:26<7:28:50, 26.96s/it]

...

0.032765310257673264
Seed: 497
Epoch 0, Treat Prop: 0.02, Loss: 0.3657
Epoch 0, Treat Prop: 0.02, Loss: 0.2264
Epoch 50, Treat Prop: 0.02, Loss: 0.0153
Epoch 50, Treat Prop: 0.02, Loss: 0.0295
Epoch 100, Treat Prop: 0.02, Loss: 0.0120
Epoch 100, Treat Prop: 0.02, Loss: 0.0157
Epoch 150, Treat Prop: 0.02, Loss: 0.0110
Epoch 150, Treat Prop: 0.02, Loss: 0.0126
Epoch 200, Treat Prop: 0.02, Loss: 0.0139
Epoch 200, Treat Prop: 0.02, Loss: 0.0149
Epoch 250, Treat Prop: 0.02, Loss: 0.0103
Epoch 250, Treat Prop: 0.02, Loss: 0.0114
Epoch 300, Treat Prop: 0.02, Loss: 0.0103
Epoch 300, Treat Prop: 0.02, Loss: 0.0112
Epoch 350, Treat Prop: 0.02, Loss: 0.0102
Epoch 350, Treat Prop: 0.02, Loss: 0.0110
0.01124163530766964
Seed: 497
Epoch 0, Treat Prop: 0.05, Loss: 0.3628
Epoch 0, Treat Prop: 0.05, Loss: 0.2670
Epoch 50, Treat Prop: 0.05, Loss: 0.0175
Epoch 50, Treat Prop: 0.05, Loss: 0.0539
Epoch 100, Treat Prop: 0.05, Loss: 0.0154
Epoch 100, Treat Prop: 0.05, Loss: 0.0237
Epoch 150, Treat Prop: 0.05

 50%|████▉     | 498/1000 [3:16:20<3:05:54, 22.22s/it]

0.02995670586824417
Seed: 498
Epoch 0, Treat Prop: 0.02, Loss: 0.3675
Epoch 0, Treat Prop: 0.02, Loss: 0.2283
Epoch 50, Treat Prop: 0.02, Loss: 0.0149
Epoch 50, Treat Prop: 0.02, Loss: 0.0304
Epoch 100, Treat Prop: 0.02, Loss: 0.0192
Epoch 100, Treat Prop: 0.02, Loss: 0.0239
Epoch 150, Treat Prop: 0.02, Loss: 0.0115
Epoch 150, Treat Prop: 0.02, Loss: 0.0162
Epoch 200, Treat Prop: 0.02, Loss: 0.0102
Epoch 200, Treat Prop: 0.02, Loss: 0.0130
Epoch 250, Treat Prop: 0.02, Loss: 0.0110
Epoch 250, Treat Prop: 0.02, Loss: 0.0140
Epoch 300, Treat Prop: 0.02, Loss: 0.0122
Epoch 300, Treat Prop: 0.02, Loss: 0.0127
Epoch 350, Treat Prop: 0.02, Loss: 0.0101
Epoch 350, Treat Prop: 0.02, Loss: 0.0116
0.011648721992969513
Seed: 498
Epoch 0, Treat Prop: 0.05, Loss: 0.3660
Epoch 0, Treat Prop: 0.05, Loss: 0.2749
Epoch 50, Treat Prop: 0.05, Loss: 0.0170
Epoch 50, Treat Prop: 0.05, Loss: 0.0542
Epoch 100, Treat Prop: 0.05, Loss: 0.0158
Epoch 100, Treat Prop: 0.05, Loss: 0.0266
Epoch 150, Treat Prop: 0.05

 50%|████▉     | 499/1000 [3:16:42<3:05:53, 22.26s/it]

0.033620528876781464
Seed: 499
Epoch 0, Treat Prop: 0.02, Loss: 0.3664
Epoch 0, Treat Prop: 0.02, Loss: 0.2283
Epoch 50, Treat Prop: 0.02, Loss: 0.0148
Epoch 50, Treat Prop: 0.02, Loss: 0.0296
Epoch 100, Treat Prop: 0.02, Loss: 0.0106
Epoch 100, Treat Prop: 0.02, Loss: 0.0166
Epoch 150, Treat Prop: 0.02, Loss: 0.0127
Epoch 150, Treat Prop: 0.02, Loss: 0.0133
Epoch 200, Treat Prop: 0.02, Loss: 0.0129
Epoch 200, Treat Prop: 0.02, Loss: 0.0130
Epoch 250, Treat Prop: 0.02, Loss: 0.0104
Epoch 250, Treat Prop: 0.02, Loss: 0.0116
Epoch 300, Treat Prop: 0.02, Loss: 0.0102
Epoch 300, Treat Prop: 0.02, Loss: 0.0114
Epoch 350, Treat Prop: 0.02, Loss: 0.0100
Epoch 350, Treat Prop: 0.02, Loss: 0.0111
0.01118226908147335
Seed: 499
Epoch 0, Treat Prop: 0.05, Loss: 0.3662
Epoch 0, Treat Prop: 0.05, Loss: 0.2741
Epoch 50, Treat Prop: 0.05, Loss: 0.0178
Epoch 50, Treat Prop: 0.05, Loss: 0.0560
Epoch 100, Treat Prop: 0.05, Loss: 0.0153
Epoch 100, Treat Prop: 0.05, Loss: 0.0279
Epoch 150, Treat Prop: 0.05

In [12]:
step_idx

6

In [ ]:
GATE_hat_list.mean(axis=0)

In [ ]:
GATE_hat_list.std(axis=0)